In [ ]:
!pip install jupyter-dash
!pip install geopandas
!pip install pygeos
!pip install dash-bootstrap-components

from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output
import dash_bootstrap_components as dbc
import pandas as pd
import geopandas as gpd
import numpy as np
import plotly.express as px
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import matplotlib.pyplot as plt
import json

**Carregando arquivo .csv com geopandas**

In [56]:
# the noise_c file is on github

df = pd.read_csv('/content/noise_c.csv')
df

,Bezirk,DB_TAG,DB_NACHT,Latitude,Longitude
0,"Aachener Straße 1, Düsseldorf, Germany",68,62,51.207592,6.776500
1,"Aachener Straße 10, Düsseldorf, Germany",68,61,51.207128,6.776436
2,"Aachener Straße 101, Düsseldorf, Germany",70,63,51.202533,6.771146
3,"Aachener Straße 103, Düsseldorf, Germany",70,62,51.202428,6.771077
4,"Aachener Straße 105, Düsseldorf, Germany",69,62,51.202332,6.771107
...,...,...,...,...,...
77013,"Zietenstraße 27, Düsseldorf, Germany",53,45,51.242225,6.776998
77014,"Zietenstraße 29, Düsseldorf, Germany",53,45,51.242274,6.777171
77015,"Zietenstraße 3, Düsseldorf, Germany",57,48,51.241556,6.774812
77016,"Zietenstraße 30, Düsseldorf, Germany",54,45,51.242552,6.776789


In [57]:
df.rename(columns={'Bezirk': 'Address'}, inplace=True)

In [58]:
df.dtypes

Address       object
DB_TAG         int64
DB_NACHT       int64
Latitude     float64
Longitude    float64
dtype: object

In [59]:
gdf = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Longitude, df.Latitude))

In [60]:
gdf.crs = "EPSG:4326"

In [61]:
gdf.shape

(77018, 6)

In [62]:
gdf.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [63]:
gdf.head()

,Address,DB_TAG,DB_NACHT,Latitude,Longitude,geometry
0,"Aachener Straße 1, Düsseldorf, Germany",68,62,51.207592,6.776500,POINT (6.77650 51.20759)
1,"Aachener Straße 10, Düsseldorf, Germany",68,61,51.207128,6.776436,POINT (6.77644 51.20713)
2,"Aachener Straße 101, Düsseldorf, Germany",70,63,51.202533,6.771146,POINT (6.77115 51.20253)
3,"Aachener Straße 103, Düsseldorf, Germany",70,62,51.202428,6.771077,POINT (6.77108 51.20243)
4,"Aachener Straße 105, Düsseldorf, Germany",69,62,51.202332,6.771107,POINT (6.77111 51.20233)


**Carregando shapefile de Düsseldorf**

In [64]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [71]:
# the Düsseldrof_Stadteile.shp file is on github

gply = gpd.read_file('/content/drive/MyDrive/map/Düsseldorf_Stadtteile.shp') 

columns_drop = ["OBJECTID", "Quelle", "Stand", "Stadtbezir", "Stadtteil", "SHAPE_Leng", "SHAPE_Area"]
gply.drop(labels = columns_drop, axis = "columns", inplace = True)

gply.rename(columns={'Name':'Neighborhood','geometry':'GEO'}, inplace=True)

In [72]:
gply

,Neighborhood,GEO
0,Wittlaer,"POLYGON Z ((753213.545 6676946.480 0.000, 7532..."
1,Angermund,"POLYGON Z ((758032.788 6682177.206 0.000, 7580..."
2,Lörick,"POLYGON Z ((750972.572 6665996.186 0.000, 7507..."
3,Heerdt,"POLYGON Z ((749803.703 6663442.957 0.000, 7496..."
4,Oberkassel,"POLYGON Z ((753486.019 6662425.887 0.000, 7534..."
5,Niederkassel,"POLYGON Z ((753437.079 6662969.087 0.000, 7528..."
6,Golzheim,"POLYGON Z ((754975.370 6664275.935 0.000, 7549..."
7,Derendorf,"POLYGON Z ((756549.378 6664501.726 0.000, 7562..."
8,Pempelfort,"POLYGON Z ((756750.257 6662093.973 0.000, 7567..."
9,Carlstadt,"POLYGON Z ((754265.923 6661187.584 0.000, 7542..."


In [73]:
gply.dtypes

Neighborhood      object
GEO             geometry
dtype: object

In [74]:
print(gply.crs)

epsg:3857


In [79]:
#gply = gply.set_geometry("GEO")
gply = gply.to_crs(4326)

gply.crs

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

In [80]:
merge=gpd.sjoin(gply,gdf,how='inner',op='contains')
merge.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:3249: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if (await self.run_code(code, result,  async_=asy)):


,Neighborhood,GEO,index_right,Address,DB_TAG,DB_NACHT,Latitude,Longitude
0,Wittlaer,"POLYGON ((6.76623 51.31355, 6.76617 51.31353, ...",20833,"Einbrunger Straße 82b, Düsseldorf, Germany",47,38,51.311149,6.754960
0,Wittlaer,"POLYGON ((6.76623 51.31355, 6.76617 51.31353, ...",28917,"Graf-Recke-Weg 8, Düsseldorf, Germany",47,39,51.312127,6.753379
0,Wittlaer,"POLYGON ((6.76623 51.31355, 6.76617 51.31353, ...",71388,"Unterdorfstraße 2a, Düsseldorf, Germany",54,45,51.309831,6.756683
0,Wittlaer,"POLYGON ((6.76623 51.31355, 6.76617 51.31353, ...",13591,"Bockumer Straße 361, Düsseldorf, Germany",60,52,51.331291,6.723766
0,Wittlaer,"POLYGON ((6.76623 51.31355, 6.76617 51.31353, ...",13521,"Bockumer Straße 263, Düsseldorf, Germany",47,39,51.328589,6.728074


In [81]:
gba = merge.groupby(['Neighborhood']).agg({'DB_TAG':'mean','DB_NACHT':'mean'})
gba.shape, gba.head()

((50, 2),                  DB_TAG   DB_NACHT
 Neighborhood                      
 Altstadt      57.699789  48.217759
 Angermund     51.446281  42.895741
 Benrath       58.261646  50.305124
 Bilk          59.179613  51.136587
 Carlstadt     60.528736  51.890805)

In [82]:
gply.set_index('Neighborhood',inplace=True)

In [83]:
gply['Daily Average']=gba['DB_TAG']
gply['Night Average']=gba['DB_NACHT']

In [84]:
gply.shape, gply.head()

((50, 3),                                                             GEO  \
 Neighborhood                                                      
 Wittlaer      POLYGON ((6.76623 51.31355, 6.76617 51.31353, ...   
 Angermund     POLYGON ((6.80952 51.34292, 6.80943 51.34251, ...   
 Lörick        POLYGON ((6.74610 51.25203, 6.74380 51.24946, ...   
 Heerdt        POLYGON ((6.73560 51.23767, 6.73399 51.23585, ...   
 Oberkassel    POLYGON ((6.76868 51.23195, 6.76868 51.23194, ...   
 
               Daily Average  Night Average  
 Neighborhood                                
 Wittlaer          53.339965      45.046811  
 Angermund         51.446281      42.895741  
 Lörick            53.602421      45.645251  
 Heerdt            60.602572      53.483601  
 Oberkassel        58.515371      50.722032  )

In [85]:
gply.reset_index(inplace=True)

In [86]:
adf=gpd.GeoDataFrame(gply)
adf.head()

,Neighborhood,GEO,Daily Average,Night Average
0,Wittlaer,"POLYGON ((6.76623 51.31355, 6.76617 51.31353, ...",53.339965,45.046811
1,Angermund,"POLYGON ((6.80952 51.34292, 6.80943 51.34251, ...",51.446281,42.895741
2,Lörick,"POLYGON ((6.74610 51.25203, 6.74380 51.24946, ...",53.602421,45.645251
3,Heerdt,"POLYGON ((6.73560 51.23767, 6.73399 51.23585, ...",60.602572,53.483601
4,Oberkassel,"POLYGON ((6.76868 51.23195, 6.76868 51.23194, ...",58.515371,50.722032


In [87]:
adf.crs = "EPSG:4326"

**Manipulando dados para construção de mapa por bairro**

In [88]:
adf.to_file("adf", driver = "GeoJSON")

with open("adf") as geofile:
  adf_hood = json.load(geofile)

In [89]:
adf_hood['features'][0].keys()

dict_keys(['type', 'properties', 'geometry'])

In [90]:
adf_hood['features'][0]['properties']

{'Neighborhood': 'Wittlaer',
 'Daily Average': 53.33996489174956,
 'Night Average': 45.04681100058514}

In [91]:
for k in range(len(adf_hood['features'])):
    adf_hood['features'][k]['id'] = k

In [92]:
adf_hood['features'][0].keys()

dict_keys(['type', 'properties', 'geometry', 'id'])

In [93]:
hood_id_map = {}
for feature in adf_hood['features']:
 feature['id'] = feature['properties']['Neighborhood']
 hood_id_map[feature['properties']['Neighborhood']] = feature['id']

**Plotando mapa por bairro**

In [94]:
fig_Day = px.choropleth_mapbox(
 adf, #soybean database
 locations = 'Neighborhood', #define the limits on the map/geography
 geojson = adf_hood, #shape information
 color = 'Daily Average', #defining the color of the scale through the database
 #hover_name = 'Avg_Day', #the information in the box
 hover_data =["Neighborhood","Daily Average"],
 #title = "Noise pollution by neighborhood in Düsseldorf (Daily average)", #title of the map
 mapbox_style = "carto-darkmatter", #defining a new map style 
 center={"lat": 51.233334, "lon": 6.783333},#define the limits that will be plotted
 zoom = 9, #map view size
 opacity = 0.5, #opacity of the map color, to appear the background
 color_continuous_scale="Viridis"
)
 
fig_Day.show()



In [ ]:
fig_Day.write_html("fig_Day.html")

In [149]:
fig_Night = px.choropleth_mapbox(
 adf, #soybean database
 locations = 'Neighborhood', #define the limits on the map/geography
 geojson = adf_hood, #shape information
 color = 'Night Average', #defining the color of the scale through the database
 #hover_name = 'Avg_Day', #the information in the box
 hover_data =["Neighborhood","Night Average"],
 #title = "Noise pollution by neighborhood in Düsseldorf (Night average)", #title of the map
 mapbox_style = "carto-darkmatter", #defining a new map style 
 center={"lat": 51.233334, "lon": 6.783333},#define the limits that will be plotted
 zoom = 9, #map view size
 opacity = 0.5, #opacity of the map color, to appear the background
 color_continuous_scale = "inferno"
)
 
fig_Night.show()

In [ ]:
fig_Night.write_html("fig_Night.html")

In [96]:
fig2 = go.Figure(layout = {"template": "plotly_dark"})
fig2.add_trace(go.Bar(x = adf['Daily Average'], 
                      y = adf['Neighborhood'], 
                      orientation='h'))
fig2.update_layout(
    yaxis = {'categoryorder':'total ascending'},
    margin = go.Margin(l=10, r=10, t=10, b=10)
)

fig2

/usr/local/lib/python3.7/dist-packages/plotly/graph_objs/_deprecations.py:410: DeprecationWarning:

plotly.graph_objs.Margin is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.layout.Margin




**Final Dashboard**

In [159]:
cols_dd = ["Daily Average", "Night Average"]

cols_aa = ["DB_TAG", "DB_NACHT"]


# Define the Figures ----------------------

fig_day = px.choropleth_mapbox(adf, 
locations = 'Neighborhood', #define the limits on the map/geography
geojson = adf_hood, #shape information
color = 'Daily Average', #defining the color of the scale through the database
color_continuous_scale = "Viridis",
hover_data = ["Neighborhood","Daily Average"],
#title = "Noise pollution by neighborhood in Düsseldorf (Night average)", #title of the map
center = {"lat": 51.233334, "lon": 6.783333},#define the limits that will be plotted
zoom = 9, #map view size
opacity = 0.6)

fig_day.update_layout(
    paper_bgcolor = "#242424",
    autosize = True,
    margin  = go.Margin(l=0, r=0, t=0, b=0),
    showlegend = False,
    mapbox_style = "carto-darkmatter"
)

fig_night = px.choropleth_mapbox(adf, 
locations = 'Neighborhood', #define the limits on the map/geography
geojson = adf_hood, #shape information
color = 'Night Average', #defining the color of the scale through the database
color_continuous_scale = "Viridis",
hover_data = ["Neighborhood","Night Average"],
#title = "Noise pollution by neighborhood in Düsseldorf (Night average)", #title of the map
center = {"lat": 51.233334, "lon": 6.783333},#define the limits that will be plotted
zoom = 9, #map view size
opacity = 0.6)

fig_night.update_layout(
    paper_bgcolor = "#242424",
    autosize = True,
    margin  = go.Margin(l=0, r=0, t=0, b=0),
    showlegend = False,
    mapbox_style = "carto-darkmatter"
)

fig_tag = px.scatter_mapbox(df, 
                        lat = "Latitude", 
                        lon = "Longitude",    
                        color = "DB_TAG",
                        zoom = 9,
                        hover_data = ["Address","DB_TAG"]  
                        )
fig_tag.update_layout(
    paper_bgcolor = "#242424",
    autosize = True,
    margin  = go.Margin(l=0, r=0, t=0, b=0),
    showlegend = False,
    mapbox_style = "carto-darkmatter")

fig_nacht = px.scatter_mapbox(df, 
                        lat = "Latitude", 
                        lon = "Longitude",    
                        color = "DB_NACHT",
                        zoom = 9,
                        hover_data = ["Address","DB_NACHT"]  
                        )
fig_nacht.update_layout(
    paper_bgcolor = "#242424",
    autosize = True,
    margin  = go.Margin(l=0, r=0, t=0, b=0),
    showlegend = False,
    mapbox_style = "carto-darkmatter")



# Layout -------------------------------------------------


app = JupyterDash(__name__, external_stylesheets = [dbc.themes.CYBORG])
app.layout = dbc.Container(
    children=[
        dbc.Row([
            html.Div([
                        html.H2(children = "Düsseldorf Noise Pollution"),
                        html.H4(children = "TechLabs Düsseldorf, Digital Shaper Program 2022"),
                        html.H6(children = "This project provides an overview of noise pollution in düsseldorf streets using different visualization technics."),
                    ], style={"background-color": "#1E1E1E", "padding": "25px"}),
            dbc.Col([
                    html.Div([
                        html.H5("1. Average noise pollution by neighborhood", style = {"margin-top": "10px"}),
                        html.Div([
                            html.P("Select the map:", style={"margin-top": "25px"}),
                            dcc.Dropdown(
                                id='demo-dropdown',
                                options=[{'label': k, 'value': k} for k in cols_dd],
                                value=cols_dd[0],
                                style={"margin-top": "10px"}
                                ),
                                html.Hr(),
                                dcc.Graph(id='display-selected-values'),
                                ]),
                              ], style={'height': '100vh', 'margin-right': '10px'}),
                    ], md=6),
            dbc.Col([
                    html.Div([
                        html.H5("2. Noise pollution by address", style = {"margin-top": "10px"}),
                        html.Div([
                            html.P("Select the map:", style={"margin-top": "25px"}),
                            dcc.Dropdown(
                                id='demo-dropdown2',
                                options=[{'label': k, 'value': k} for k in cols_aa],
                                value=cols_aa[0],
                                style={"margin-top": "10px"}
                                ),
                                html.Hr(),
                                dcc.Graph(id='display-selected-values2'),
                                ]),
                              ], style={'height': '100vh', 'margin-right': '10px'}),
                    ], md=6),     
                 ],
                )], fluid=True)

@app.callback(Output('display-selected-values', 'figure'),
    [Input('demo-dropdown', 'value')])
def update_output(value):
  # Show the right figure based on the input
  if value == "Daily Average":
    return fig_day
  else:
    return fig_night

@app.callback(Output('display-selected-values2', 'figure'),
    [Input('demo-dropdown2', 'value')])
def update_output(value):
  # Show the right figure based on the input
  if value == "DB_TAG":
    return fig_tag
  else:
    return fig_nacht

if __name__ == '__main__':
    app.run_server()

Dash app running on:


<IPython.core.display.Javascript object>

In [ ]:
!pip install pyngrok